# Linear Operators

In [1]:
%pylab inline
import scipy as sp
import scipy.sparse as sparse
import scipy.sparse.linalg as sla

Populating the interactive namespace from numpy and matplotlib


In linear algebra, a linear transformation, linear operator, or linear map, is a map of vector spaces $T:V \to W$ where 
$$T(\alpha v_1 + \beta v_2) = \alpha T v_1 + \beta T v_2$$

If you choose bases for the vector spaces $V$ and $W$, you can represent $T$ using a (dense) matrix.  However, there are many situations where we may want to represent $T$ in some other format which will allow us to do faster matrix-vector and matrix-matrix multiplications.

The case of a sparse matrix is handled by [special matrix formats](sparse.html), but there are also situations in which dense matrices can also be applied quickly.

## Low Rank Matrices

The easiest situation to describe is a low-rank matrix.  We saw an example of this when we first saw object-oriented programming in the [`LinearMap` class](https://caam37830.github.io/book/00_python/classes.html#class-inheritance). SciPy provides a very similar class which can be used to construct arbitrary linear operators, which is called `LinearOperator`.

The `LinearOperator` class can be found in `scipy.sparse.linalg`.  The `aslinearoperator` function lets us to treat dense and sparse arrays as `LinearOperators`.

In [2]:
from scipy.sparse.linalg import LinearOperator, aslinearoperator

As an example, let's construct a `LinearOperator` that acts as the matrix of all ones. This matrix is rank-1 and can be written as $11^T$, where $1$ is a vector of the appropriate dimension.

In [3]:
n = 20
m = 10
onesn = aslinearoperator(np.ones((n,1)))
onesm = aslinearoperator(np.ones((m,1)))
A = onesm @ onesn.T

In [4]:
v = np.random.randn(n)
A @ v

array([6.68543888, 6.68543888, 6.68543888, 6.68543888, 6.68543888,
       6.68543888, 6.68543888, 6.68543888, 6.68543888, 6.68543888])

Let's do a timing comparison between the `LinearOperator` and a dense matrix.

In [5]:
n = 2000
m = 1000
onesn = aslinearoperator(np.ones((n,1)))
onesm = aslinearoperator(np.ones((m,1)))
Alo = onesm @ onesn.T
Ad = np.ones((m,n)) # dense array of all ones

In [6]:
import time

x = np.random.randn(n)
y = np.empty(m)

t0 = time.time()
y = Alo @ x
t1 = time.time()
print("time for linear operator: {} sec.".format(t1 - t0))
tlo = t1 - t0

t0 = time.time()
y = Ad @ x
t1 = time.time()
print("time for dense: {} sec.".format(t1 - t0))
td = t1 - t0

print("LinearOperator speedup = {} x".format(td / tlo))


time for linear operator: 0.00013518333435058594 sec.
time for dense: 0.0014600753784179688 sec.
LinearOperator speedup = 10.800705467372135 x


## Linear Operators from Functions

We can also specify linear operators through functions.

Another way to characterize the action of a matrix containing all 1s is that it sums up all the entries in a vector of length `n`, and repeats the sum in every entry in a vector of length `m`.  For simplicity, we'll assume that `m = n`.

In [7]:
# a function that sums the entries in a vector and repeats the sum in every entry of a vector of the same shape.
# works on columns of matrices as well
A = lambda X : np.sum(X, axis=0).reshape(1,-1).repeat(X.shape[0], axis=0)

In [8]:
x = np.random.rand(5)
A(x)

array([[3.32723506],
       [3.32723506],
       [3.32723506],
       [3.32723506],
       [3.32723506]])

In [9]:
np.sum(x)

3.3272350624839993

The problem is that this function doesn't play nicely with other linear operators.  In order to do that, we wrap the function in the `LinearOperator` class.  For a `LinearOperator` `A`, We define the functions
* `matvec` (computes `A @ x`)
* `rmatvec` (computes `A.T @ x`)
* `matmat` (computes `A @ B`)
* `rmatmat` (computes `A.T @ B`)
as well as the shape of the operator.  Note that you don't need to define all the functions (`matvec` is most important), but you may get errors in certain situations (e.g. taking transpose) if you don't.

In [10]:
# works on square matrices
Afun = lambda X : np.sum(X, axis=0).reshape(1,-1).repeat(X.shape[0], axis=0)

m = 10 # linear operator of size 10

A = LinearOperator(
    shape   = (m,m),
    matvec  = Afun,
    rmatvec = Afun,
    matmat  = Afun,
    rmatmat = Afun,
    dtype=float   
)

In [11]:
A @ np.random.rand(m)

array([6.06306004, 6.06306004, 6.06306004, 6.06306004, 6.06306004,
       6.06306004, 6.06306004, 6.06306004, 6.06306004, 6.06306004])

## Composition

Because of linearity, sums and products of Linear operators can also have nice properties.  This is encoded in the `LinearOperator` class:

In [12]:
B1 = A + A # acts as the matrix with 2 in every entry
B2 = A @ A # acts that the matrix with m in every entry

In [13]:
B1 @ np.random.rand(m)

array([12.0390204, 12.0390204, 12.0390204, 12.0390204, 12.0390204,
       12.0390204, 12.0390204, 12.0390204, 12.0390204, 12.0390204])

In [14]:
B2 @ np.random.rand(m)

array([51.19468737, 51.19468737, 51.19468737, 51.19468737, 51.19468737,
       51.19468737, 51.19468737, 51.19468737, 51.19468737, 51.19468737])

## Exercises

1. Define a `LinearOperator` that "centers" a vector: `A: x -> x - mean(x)`. i.e. we subtract the mean of the vector from every entry of the vector.

In [15]:
## Your code here
Afun = lambda x : x - np.sum(x)/np.size(x)


m = 10 # linear operator of size 10

A = LinearOperator(
    shape   = (m,m),
    matvec  = Afun,

    dtype= float   
)

np.size(np.random.rand(m))

np.sum(A @ np.random.rand(m))

1.1102230246251565e-16

2. Define a `LinearOperator` that returns the differences in a vector.  i.e. `A` is a `n-1 x n` operator, where `(A @ x)[i]` is `x[i+1] - x[i]`.  You may want to look at `np.diff`.

In [16]:
## Your code here


3. In both the above exercises, you could define these linear operators using either functions or a combination of dense/sparse matrices.  If you completed an exercise above using a function, write a second version which uses dense/sparse matrices.  If you completed an exercise above using dense/sparse matrices, write a second version that uses functions.

In [17]:
## Your code here


4. Define a linear operator that applies $A^{-1}$ to a vector $b$ (i.e. $A^{-1} b$. Don't use matrix inverse!):

In [20]:
import numpy.linalg as la

m  = 10
AA = np.random.randn(m, m)

#def wrapsolve(A, x):
#    return la.solve(A, x)

## Your code here
Afun  = lambda x : la.solve(AA, x)
AfunT = lambda x : la.solve(AA.T, x)
m = 10 # linear operator of size 10
A = LinearOperator(
    shape   = (m,m),
    matvec  = Afun,
    rmatvec = AfunT,
    matmat  = Afun,
    rmatmat = AfunT,
    dtype= float
)

A @ np.random.rand(m,m)



array([[ 0.47450663, -0.08634583,  0.15336575, -0.01344204,  0.14684916,
         0.47171607,  0.46737918,  0.46426221, -0.04305705,  0.6266941 ],
       [ 0.22969871,  0.57790199,  0.23740934,  0.24618601,  0.20442024,
         0.2219935 ,  0.15752819,  0.12281025,  0.21517846, -0.0491611 ],
       [ 0.18390889,  0.04779724,  0.20547745,  0.12346904,  0.27571138,
         0.14180424,  0.0230146 ,  0.5737584 ,  0.03751337,  1.07821882],
       [-0.11315195, -0.27121212, -0.40216917, -0.44990773, -0.27183837,
        -0.16077701, -0.30286618, -0.41012491, -0.37327944, -0.42185135],
       [-0.26926795, -0.14715563, -0.0791498 , -0.45419682, -0.35490079,
        -0.34668126, -0.24076403, -0.24679995, -0.16737936, -0.08367607],
       [-0.5308723 , -0.00583382, -0.40730556, -0.39952925, -0.50956389,
        -0.33292204, -0.31752605, -0.98400867, -0.38529858, -1.32174258],
       [-0.22124424, -0.27186632, -0.13400655, -0.25827762, -0.18090092,
        -0.25119043, -0.28054817, -0.2258422 